<a href="https://colab.research.google.com/github/ANJUMNAIR/SPARKS-INTERNSHIP-TASK/blob/main/Queros_Coffe_house_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [106]:
df = pd.read_excel("/content/coffee-house-satisfactory-survey.xlsx")

In [107]:
df.head()

,Timestamp,1. Your Gender,2. Your Age,3. Are you currently....?,4. What is your annual income?,5. How often do you visit Coffee House?,6. How do you usually enjoy Coffee House?,7. How much time do you normally spend during your visit?,8. The nearest Coffee House's outlet to you is...?,9. Do you have Coffee House membership card?,...,"11. On average, how much would you spend at Coffee House per visit?","12. How would you rate the quality of Coffee House compared to other brands (Coffee Bean, Old Town White Coffee..) to be:",13. How would you rate the price range at Coffee House?,14. How important are sales and promotions in your purchase decision?,"15. How would you rate the ambiance at Coffee House? (lighting, music, etc...)",16. You rate the WiFi quality at Coffee House as..,"17. How would you rate the service at Coffee House? (Promptness, friendliness, etc..)",18. How likely you will choose Coffee House for doing business meetings or hangout with friends?,19. How do you come to hear of promotions at Coffee House? Check all that apply.,20. Will you continue buying Coffee House?
0,2019/10/01 12:38:43 PM GMT+8,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Dine in,Between 30 minutes to 1 hour,within 1km,Yes,...,Less than RM20,4,3,5,5,4,4,3,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes
1,2019/10/01 12:38:54 PM GMT+8,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Take away,Below 30 minutes,1km - 3km,Yes,...,Less than RM20,4,3,4,4,4,5,2,Social Media;In Store displays,Yes
2,2019/10/01 12:38:56 PM GMT+8,Male,From 20 to 29,Employed,"Less than RM25,000",Monthly,Dine in,Between 30 minutes to 1 hour,more than 3km,Yes,...,Less than RM20,4,3,4,4,4,4,3,In Store displays;Billboards,Yes
3,2019/10/01 12:39:08 PM GMT+8,Female,From 20 to 29,Student,"Less than RM25,000",Rarely,Take away,Below 30 minutes,more than 3km,No,...,Less than RM20,2,1,4,3,3,3,3,Through friends and word of mouth,No
4,2019/10/01 12:39:20 PM GMT+8,Male,From 20 to 29,Student,"Less than RM25,000",Monthly,Take away,Between 30 minutes to 1 hour,1km - 3km,No,...,Around RM20 - RM40,3,3,4,2,2,3,3,Starbucks Website/Apps;Social Media,Yes


In [108]:
df.columns

Index(['Timestamp', '1. Your Gender', '2. Your Age',
       '3. Are you currently....?', '4. What is your annual income?',
       '5. How often do you visit Coffee House?',
       '6. How do you usually enjoy Coffee House?',
       '7. How much time do you normally  spend during your visit?',
       '8. The nearest Coffee House's outlet to you is...?',
       '9. Do you have Coffee House membership card?',
       '10. What do you most frequently purchase at Coffee House?',
       '11. On average, how much would you spend at Coffee House per visit?',
       '12. How would you rate the quality of Coffee House compared to other brands (Coffee Bean, Old Town White Coffee..) to be:',
       '13. How would you rate the price range at Coffee House?',
       '14. How important are sales and promotions in your purchase decision?',
       '15. How would you rate the ambiance at Coffee House? (lighting, music, etc...)',
       '16. You rate the WiFi quality at Coffee House as..',
       '17. How 

In [109]:
df.isnull().sum()

Timestamp                                                                                                                    0
1. Your Gender                                                                                                               0
2. Your Age                                                                                                                  0
3. Are you currently....?                                                                                                    0
4. What is your annual income?                                                                                               0
5. How often do you visit Coffee House?                                                                                      0
6. How do you usually enjoy Coffee House?                                                                                    1
7. How much time do you normally  spend during your visit?                                                     

#Define the new heasders

In [110]:
new_headers = [
    'Timestamp',
    'Gender',
    'Age',
    'Current status',
    'Annual Income',
    'Visit Frequency',
    'Enjoyment Style',
    'Visit Duration',
    'Nearest Outlet',
    'Membership Card',
    'Frequently Purchased Items',
    'Average Spending',
    'Quality Rating',
    'Price Rating',
    'Promotion Importance',
    'Wifi Rating',
    'Ambience Rating',
    'Service Rating',
    'Meeting Likelihood',
    'Promotion Awareness',
    'Continue Purchasing'
]

#Assign the new headers to the data frame

In [111]:
new_df = df

In [112]:
new_df.columns = new_headers

In [113]:
new_df.shape

(122, 21)

In [114]:
new_df = new_df.drop(columns=['Timestamp'])

In [115]:
new_df['Continue Purchasing'].value_counts()

Yes    94
No     28
Name: Continue Purchasing, dtype: int64

In [116]:
new_df.dtypes


Gender                        object
Age                           object
Current status                object
Annual Income                 object
Visit Frequency               object
Enjoyment Style               object
Visit Duration                object
Nearest Outlet                object
Membership Card               object
Frequently Purchased Items    object
Average Spending              object
Quality Rating                 int64
Price Rating                   int64
Promotion Importance           int64
Wifi Rating                    int64
Ambience Rating                int64
Service Rating                 int64
Meeting Likelihood             int64
Promotion Awareness           object
Continue Purchasing           object
dtype: object

In [117]:
# Make a copy of the original DataFrame
transformed_df = df.copy()
# Encode "Gender" to binary values (1 for Male, 0 for Female)
transformed_df['Gender (Binary)'] = transformed_df['Gender'].map({'Male': 1, 'Female': 0})
# Encode "Continue Purchasing" to binary values (1 for Yes, 0 for No)
transformed_df['Continue Purchasing (Binary)'] = transformed_df['Continue Purchasing'].map({'Yes': 1, 'No': 0})
# Now, 'over_transformed_df' contains the original data with "Gender" and "Continue Purchasing" columns binary

In [118]:
# Encode the "Age" column into numerical values using midpoint approach
transformed_df['Age'] = transformed_df['Age'].map({
    'Below 20': 20,
    'From 20 to 29': 25,
    'From 30 to 39': 35,
    '40 and above': 45
})

In [119]:
# Encode the "Annual Income" column into numerical values using midpoint approach
transformed_df['Annual Income'] = transformed_df['Annual Income'].map({
    'Less than RM25,000': 20000,
    'RM25,000 - RM50,000': 37500,
    'RM50,000 - RM100,000': 75000,
    'RM100,000 - RM150,000': 125000,
    'More than RM150,000': 175000
})

In [120]:
# Create a function to convert visit frequency to the number of days
def convert_visit_frequency_to_days(frequency):
    if frequency == 'Never':
        return 0
    elif frequency == 'Rarely':
        return 1 # You can assign a specific value for "Rarely" based on your definition
    elif frequency == 'Monthly':
        return 30
    elif frequency == 'Weekly':
        return 7
    elif frequency == 'Daily':
        return 1
    else:
        return None # Handle any other values as needed
# Apply the conversion function to create the "Visit Frequency Days" column
transformed_df['Visit Frequency Days'] = transformed_df['Visit Frequency'].apply(convert_visit_frequency_to_days)
# You can drop the original "Visit Frequency" column if you don't need it
transformed_df.drop(columns=['Visit Frequency'], inplace=True)

In [121]:
# Create a function to convert visit duration to minutes
def convert_visit_duration_to_minutes(duration):
    if duration == 'Below 30 minutes':
        return 15
    elif duration == 'Between 30 minutes to 1 hour':
        return 45
    elif duration == 'Between 1 hour to 2 hours':
        return 90
    elif duration == 'Between 2 hours to 3 hours':
        return 150
    elif duration == 'More than 3 hours':
        return 210
    else:
        return None # Handle any other values as needed
# Apply the conversion function to create the "Visit Duration (minutes)" column
transformed_df['Visit Duration (minutes)'] = transformed_df['Visit Duration'].apply(convert_visit_duration_to_minutes)
# You can drop the original "Visit Duration" column if you don't need it
transformed_df.drop(columns=['Visit Duration'], inplace=True)


In [122]:
# Create a function to convert nearest outlet distance to kilometers
def convert_nearest_outlet_distance_to_km(distance):
    if distance == 'within 1km':
        return 0.5 # Midpoint within 1km
    elif distance == '1km - 3km':
        return 2.0 # Midpoint between 1km and 3km
    elif distance == 'more than 3km':
        return 4.0 # Midpoint for more than 3km
    else:
        return None # Handle any other values as needed
# Apply the conversion function to create the "Distance to Nearest Outlet (km)" column
transformed_df['Distance to Nearest Outlet (km)'] = transformed_df['Nearest Outlet'].apply(convert_nearest_outlet_distance_to_km)
# You can drop the original "Nearest Outlet" column if you don't need it
transformed_df.drop(columns=['Nearest Outlet'], inplace=True)


In [123]:
# Create a function to convert "Membership Card" to binary values
def convert_membership_card(card):
    if card == 'Yes':
        return 1
    elif card == 'No':
        return 0
    else:
        return None # Handle any other values as needed
# Apply the conversion function to create the "Membership Card (Binary)" column
transformed_df['Membership Card (Binary)'] = transformed_df['Membership Card'].apply(convert_membership_card)
# You can drop the original "Membership Card" column if you don't need it
transformed_df.drop(columns=['Membership Card'], inplace=True)

In [124]:
# Create a function to convert average spending to numerical values (RM)
def convert_average_spending_to_rm(spending):
    if spending == 'Less than RM20':
        return 20
    elif spending == 'Around RM20 - RM40':
        return 30
    elif spending == 'Zero':
        return 0
    elif spending == 'More than RM40':
        return 50
    else:
        return None # Handle any other values as needed
# Apply the conversion function to create the "Average Spending (RM)" column
transformed_df['Average Spending (RM)'] = transformed_df['Average Spending'].apply(convert_average_spending_to_rm)
# You can drop the original "Average Spending" column if you don't need it
transformed_df.drop(columns=['Average Spending'], inplace=True)


In [125]:
transformed_df.drop(columns=['Timestamp'], inplace=True)


In [126]:
transformed_df.dtypes

Gender                              object
Age                                  int64
Current status                      object
Annual Income                        int64
Enjoyment Style                     object
Frequently Purchased Items          object
Quality Rating                       int64
Price Rating                         int64
Promotion Importance                 int64
Wifi Rating                          int64
Ambience Rating                      int64
Service Rating                       int64
Meeting Likelihood                   int64
Promotion Awareness                 object
Continue Purchasing                 object
Gender (Binary)                      int64
Continue Purchasing (Binary)         int64
Visit Frequency Days                 int64
Visit Duration (minutes)             int64
Distance to Nearest Outlet (km)    float64
Membership Card (Binary)             int64
Average Spending (RM)                int64
dtype: object

In [127]:
transformed_df.isnull().sum()

Gender                             0
Age                                0
Current status                     0
Annual Income                      0
Enjoyment Style                    1
Frequently Purchased Items         0
Quality Rating                     0
Price Rating                       0
Promotion Importance               0
Wifi Rating                        0
Ambience Rating                    0
Service Rating                     0
Meeting Likelihood                 0
Promotion Awareness                1
Continue Purchasing                0
Gender (Binary)                    0
Continue Purchasing (Binary)       0
Visit Frequency Days               0
Visit Duration (minutes)           0
Distance to Nearest Outlet (km)    0
Membership Card (Binary)           0
Average Spending (RM)              0
dtype: int64

In [128]:
transformed_df


,Gender,Age,Current status,Annual Income,Enjoyment Style,Frequently Purchased Items,Quality Rating,Price Rating,Promotion Importance,Wifi Rating,...,Meeting Likelihood,Promotion Awareness,Continue Purchasing,Gender (Binary),Continue Purchasing (Binary),Visit Frequency Days,Visit Duration (minutes),Distance to Nearest Outlet (km),Membership Card (Binary),Average Spending (RM)
0,Female,25,Student,20000,Dine in,Coffee,4,3,5,5,...,3,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes,0,1,1,45,0.5,1,20
1,Female,25,Student,20000,Take away,Cold drinks;Pastries,4,3,4,4,...,2,Social Media;In Store displays,Yes,0,1,1,15,2.0,1,20
2,Male,25,Employed,20000,Dine in,Coffee,4,3,4,4,...,3,In Store displays;Billboards,Yes,1,1,30,45,4.0,1,20
3,Female,25,Student,20000,Take away,Coffee,2,1,4,3,...,3,Through friends and word of mouth,No,0,0,1,15,4.0,0,20
4,Male,25,Student,20000,Take away,Coffee;Sandwiches,3,3,4,2,...,3,Starbucks Website/Apps;Social Media,Yes,1,1,30,45,2.0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Male,45,Self-employed,37500,Dine in,Coffee,3,3,5,3,...,4,Starbucks Website/Apps;Social Media,Yes,1,1,30,90,2.0,1,30
118,Male,25,Employed,20000,Dine in,Coffee;Cold drinks;Juices;Pastries;Sandwiches,5,5,5,5,...,5,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes,1,1,30,90,2.0,1,50
119,Male,25,Student,20000,Dine in,Coffee;Cold drinks,3,2,4,3,...,4,Social Media;Through friends and word of mouth...,No,1,0,1,45,2.0,0,20
120,Female,25,Employed,20000,Take away,Coffee,4,4,4,4,...,4,Social Media;Through friends and word of mouth...,Yes,0,1,1,15,0.5,0,20


In [129]:
# Separate categorical and numerical columns
categorical_columns = transformed_df.select_dtypes(include=['object']).columns
numerical_columns = transformed_df.select_dtypes(exclude=['object']).columns
# Create the categorical dataset
categorical_data = transformed_df[categorical_columns]
# Create the numerical dataset
numerical_data = transformed_df[numerical_columns]

In [130]:
categorical_data

,Gender,Current status,Enjoyment Style,Frequently Purchased Items,Promotion Awareness,Continue Purchasing
0,Female,Student,Dine in,Coffee,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes
1,Female,Student,Take away,Cold drinks;Pastries,Social Media;In Store displays,Yes
2,Male,Employed,Dine in,Coffee,In Store displays;Billboards,Yes
3,Female,Student,Take away,Coffee,Through friends and word of mouth,No
4,Male,Student,Take away,Coffee;Sandwiches,Starbucks Website/Apps;Social Media,Yes
...,...,...,...,...,...,...
117,Male,Self-employed,Dine in,Coffee,Starbucks Website/Apps;Social Media,Yes
118,Male,Employed,Dine in,Coffee;Cold drinks;Juices;Pastries;Sandwiches,Starbucks Website/Apps;Social Media;Emails;Dea...,Yes
119,Male,Student,Dine in,Coffee;Cold drinks,Social Media;Through friends and word of mouth...,No
120,Female,Employed,Take away,Coffee,Social Media;Through friends and word of mouth...,Yes


In [131]:
numerical_data

,Age,Annual Income,Quality Rating,Price Rating,Promotion Importance,Wifi Rating,Ambience Rating,Service Rating,Meeting Likelihood,Gender (Binary),Continue Purchasing (Binary),Visit Frequency Days,Visit Duration (minutes),Distance to Nearest Outlet (km),Membership Card (Binary),Average Spending (RM)
0,25,20000,4,3,5,5,4,4,3,0,1,1,45,0.5,1,20
1,25,20000,4,3,4,4,4,5,2,0,1,1,15,2.0,1,20
2,25,20000,4,3,4,4,4,4,3,1,1,30,45,4.0,1,20
3,25,20000,2,1,4,3,3,3,3,0,0,1,15,4.0,0,20
4,25,20000,3,3,4,2,2,3,3,1,1,30,45,2.0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,45,37500,3,3,5,3,2,4,4,1,1,30,90,2.0,1,30
118,25,20000,5,5,5,5,5,5,5,1,1,30,90,2.0,1,50
119,25,20000,3,2,4,3,3,3,4,1,0,1,45,2.0,0,20
120,25,20000,4,4,4,4,4,4,4,0,1,1,15,0.5,0,20


In [132]:
numerical_data.dtypes

Age                                  int64
Annual Income                        int64
Quality Rating                       int64
Price Rating                         int64
Promotion Importance                 int64
Wifi Rating                          int64
Ambience Rating                      int64
Service Rating                       int64
Meeting Likelihood                   int64
Gender (Binary)                      int64
Continue Purchasing (Binary)         int64
Visit Frequency Days                 int64
Visit Duration (minutes)             int64
Distance to Nearest Outlet (km)    float64
Membership Card (Binary)             int64
Average Spending (RM)                int64
dtype: object